In [ ]:
# !cp -r logs logs_151121_fix_pos_emb_search_param
# !cp -r checkpoints checkpoints_151121_fix_pos_emb_search_param

In [1]:
# !tar -zcvf checkpoints.tar.gz checkpoints
# !mv checkpoints.tar.gz checkpoints_101121_no_pos_emb.tar.gz

# ! tar -zcvf logs_101121_no_pos_emb.tar.gz logs

In [ ]:
# !rm -r logs
# !rm -r checkpoints

In [37]:
# !mkdir logs
# !mkdir checkpoints

In [1]:
import numpy as np
import random
import time
import torch
from x_transformers.x_transformers import XTransformer
import torch

from run_experiment import *
from generate_data import *

## Variables

In [2]:
from sklearn.model_selection import ParameterGrid

TAG = '10tkn_len24_ext'

TASK_NAME = 'copy'
TRAIN_SIZE = 100_000
VAL_SIZE = 2_000
TEST_SIZE = 10_000
NUM_INITS = 4


NUM_BATCHES = int(4e5)
BATCH_SIZE = 128
GENERATE_EVERY  = 10000
NUM_TOKENS = 10 + 2
ENC_SEQ_LEN = 24
DEC_SEQ_LEN = 48

INPUT_LEN = 24

#### Generate data

In [3]:
# np.random.seed(42)

# class copy_generator:
#     def __init__(self):
#         self.src_mask = torch.ones(BATCH_SIZE, ENC_SEQ_LEN).bool().cuda()
#         self.tgt_mask = torch.ones(BATCH_SIZE, DEC_SEQ_LEN+1).bool().cuda()
    
#     def __next__(self):
#         X = np.zeros([BATCH_SIZE, ENC_SEQ_LEN]).astype(int)
#         y = np.zeros([BATCH_SIZE, DEC_SEQ_LEN+1]).astype(int)
#         y[:, 0] = 1
#         for i in range(BATCH_SIZE):
#             sequence_length = ENC_SEQ_LEN
#             random_sequence = np.random.randint(2, NUM_TOKENS, sequence_length)
            
#             X[i, :sequence_length] = random_sequence
#             y[i, 1: 2 * sequence_length + 1] = np.concatenate([random_sequence] * 2)

#         return torch.tensor(X), torch.tensor(y), self.src_mask, self.tgt_mask 
    
# generator = copy_generator()
# generate_data(generator, path=f'data{INPUT_LEN}', task_name=TASK_NAME, train_size=TRAIN_SIZE, test_size=TEST_SIZE, val_size=VAL_SIZE, batch_size=BATCH_SIZE)  

### Run

In [5]:
LEARNING_RATE = 0.0007

model_parameters = ParameterGrid({'dim': [128],
    'tie_token_embeds': [True],
    'return_tgt_loss': [True],
    'enc_num_tokens': [NUM_TOKENS],
    'depth,heads': [(2, 4)],
    'enc_max_seq_len': [24],
    'dec_num_tokens': [NUM_TOKENS],
    'dec_max_seq_len': [DEC_SEQ_LEN],
    'enc_num_memory_tokens': [2, 8, 0]})

print('Total runs: ', NUM_INITS * len(model_parameters))

Total runs:  12


In [ ]:
gen_train = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE)
gen_val = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE)
gen_test = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE)


print_file = f'logs/{TASK_NAME}_{TAG}_memory_logs2.txt'
t = time.time()
with torch.cuda.device(1):
    for init_num in range(NUM_INITS):
        with open(print_file, 'a') as f:
            f.write('\n\nInit number ' + str(init_num)+'\n')
        for i, param in enumerate(list(model_parameters)):
            with open(print_file, 'a') as f:
                f.write('\n\n' + str(param)+'\n')
            param['enc_depth'], param['enc_heads'] = param['depth,heads']
            param['dec_depth'], param['dec_heads'] = param['depth,heads']
            param.pop('depth,heads')

            with open(print_file, 'a') as f:
                f.write(f'{i / len(model_parameters) * 100}%')
            model = XTransformer(**param).cuda()

            model_name = f"{TASK_NAME}{INPUT_LEN}_dim{param['dim']}d{param['enc_depth']}h{param['enc_heads']}M{param['enc_num_memory_tokens']}l{param['enc_max_seq_len']}_{TAG}_v{init_num}"

            optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
            train_validate_model(model, 
                            train_generator=gen_train, 
                            val_generator=gen_val, 
                            optim=optim, 
                            model_name=model_name, 
                            config=param,
                            num_batches=NUM_BATCHES,
                            generate_every=GENERATE_EVERY,
                            print_file=print_file,
                            tag=TAG,
                            overfit_stop=False)
            test_model(model, gen_test, model_name, param, TASK_NAME, tag=TAG)
            with open(print_file, 'a') as f:
                f.write(f'\nTotal time: {time.time() - t}\n')
            t = time.time()

mkdir: 

### Refit models

In [6]:
# import os

# def load_cpt(config, v, task_name, input_length):
#     for fns in os.walk('checkpoints'):
#         model_names = fns[2]
        
#     prefix = '{task_name}_dim{dim}d{d}h{h}M{M}l{l}'
#     name = prefix.format(task_name=task_name,
#                         dim=config['dim'],
#                         d=config['enc_depth'], h=config['enc_heads'], 
#                         M=config['enc_num_memory_tokens'], 
#                         l=input_length)

#     checkpoint_paths = ['checkpoints/' + n for n in model_names if name in n]
#     cpt = torch.load(checkpoint_paths[v])
#     bn, model_state, optim_state = cpt['batch_num'], cpt['state_dict'], cpt['optimizer']

#     model = XTransformer(**config).cuda()
#     model.load_state_dict(model_state)

#     optim = torch.optim.Adam(model.parameters(), lr=0.001)
#     optim.load_state_dict(optim_state)

#     return bn, model, optim


In [7]:
# TAG = 'refit_to_max'
# LEARNING_RATE = 0.001

# path = f"checkpoints/{TASK_NAME}{INPUT_LEN}/"

# for name in next(os.walk(path))[2]:
#     print(name)
#     if name == 'copy24_dim128d2h4M12l12_10tkn_len24_v2_10tkn_len24.pt':
#         continue
#     cpt = torch.load(path+name)
#     print(cpt['batch_num'])
#     delta_batches = NUM_BATCHES - cpt['batch_num'] - 1
#     if delta_batches < 1:
#         continue
    
#     split = name.split('_')
#     config = {'dec_max_seq_len': DEC_SEQ_LEN,
#          'dec_num_tokens': NUM_TOKENS,
#          'dim': int(split[1].split('dim')[1].split('d')[0]),
#          'enc_max_seq_len': int(split[1].split('M')[1].split('l')[1]),
#          'enc_num_memory_tokens': int(split[1].split('M')[1].split('l')[0]),
#          'enc_num_tokens': NUM_TOKENS,
#          'return_tgt_loss': True,
#          'tie_token_embeds': True,
#          'enc_depth': int(split[1][3:].split('d')[1].split('h')[0]),
#          'enc_heads': int(split[1][3:].split('d')[1].split('h')[1].split('M')[0]),
#          'dec_depth': int(split[1][3:].split('d')[1].split('h')[0]),
#          'dec_heads': int(split[1][3:].split('d')[1].split('h')[1].split('M')[0]),
#          'tag': TAG,
#          'task_name': TASK_NAME}
    
    
#     gen_train = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE)
#     gen_val = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE)
#     gen_test = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE)


#     print_file = f'logs/{TASK_NAME}_{TAG}_memory_logs.txt'
#     t = time.time()
#     with torch.cuda.device(0):
#         with open(print_file, 'a') as f:
#             f.write('\n\n' + str(config)+'\n')
#             f.write(str(delta_batches) + ' batches to go.\n')

#         print('\n\n' + str(config)+'\n')
#         print(str(delta_batches) + ' batches to go.\n')
#         model_name = name
#         model = XTransformer(**config).cuda()
#         optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
        
#         model.load_state_dict(cpt['state_dict'])
#         optim.load_state_dict(cpt['optimizer'])

#         train_validate_model(model, 
#                             train_generator=gen_train, 
#                             val_generator=gen_val, 
#                             optim=optim, 
#                             model_name=model_name, 
#                             config=config,
#                             num_batches=delta_batches,
#                             generate_every=GENERATE_EVERY,
#                             print_file=print_file,
#                             tag=TAG,
#                             overfit_stop=False)
#         test_model(model, gen_test, model_name, config, TASK_NAME, tag=TAG)

#         with open(print_file, 'a') as f:
#             f.write(f'\nTotal time: {time.time() - t}\n')
#         t = time.time()

In [ ]:
test_model(model, gen_test, model_name, config, TASK_NAME, tag=TAG)

In [ ]:
# gen_train = data_loader(task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE, enc_seq_len=INPUT_LEN, dec_seq_len=DEC_SEQ_LEN)
# gen_val = data_loader(task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE, enc_seq_len=INPUT_LEN, dec_seq_len=DEC_SEQ_LEN)
# gen_test = data_loader(task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE, enc_seq_len=INPUT_LEN, dec_seq_len=DEC_SEQ_LEN)


# print_file = f'logs/{TASK_NAME}_{TAG}_memory_logs.txt'
# t = time.time()
# with torch.cuda.device(0):
#     for init_num in range(NUM_INITS):
#         with open(print_file, 'a') as f:
#             f.write('\n\nInit number ' + str(init_num)+'\n')
#         for i, param in enumerate(list(model_parameters)):
#             with open(print_file, 'a') as f:
#                 f.write('\n\n' + str(param)+'\n')
#             param['enc_depth'], param['enc_heads'] = param['depth,heads']
#             param['dec_depth'], param['dec_heads'] = param['depth,heads']
#             param.pop('depth,heads')

#             with open(print_file, 'a') as f:
#                 f.write(f'{i / len(model_parameters) * 100}%')
#             model = XTransformer(**param).cuda()

#             model_name = f"{TASK_NAME}{INPUT_LEN}_dim{param['dim']}d{param['enc_depth']}h{param['enc_heads']}M{param['enc_num_memory_tokens']}l{param['enc_max_seq_len']}_v{init_num}"

#             optim = torch.optim.Adam(model.ффparameters(), lr=LEARNING_RATE)
            
#             bn, model, optim = load_cpt(param, v=init_num, task_name='copy55', input_length=param['enc_max_seq_len'])
#             with open(print_file, 'a') as f:
#                 f.write(f'BN: {bn}\n')
#             if bn < 130_000:
#                 train_validate_model(model, 
#                                     train_generator=gen_train, 
#                                     val_generator=gen_val, 
#                                     optim=optim, 
#                                     model_name=model_name, 
#                                     dec_seq_len=DEC_SEQ_LEN,
#                                     num_batches=NUM_BATCHES,
#                                     generate_every=GENERATE_EVERY,
#                                     print_file=print_file,
#                                     tag=TAG,
#                                     overfit_stop=False,
#                                     head_start=(130_000 - bn)/GENERATE_EVERY)
#                 test_model(model, gen_test, model_name, param, TASK_NAME, tag=TAG, dec_seq_len=param['dec_max_seq_len'])
#             with open(print_file, 'a') as f:
#                 f.write(f'\nTotal time: {time.time() - t}\n')
#             t = time.time()

In [ ]:
from run_experiment import save_checkpoint

In [ ]:
# save_path = f'checkpoints/{model_name}_b{i}_{TAG}_maxval.pt'
# save_cpt(save_path, model, optim)

# if i // generate_every < head_start:
#     continue

# # early stopping
# smoothed_val_scores = [np.mean(validation_scores[i-WINDOW_SIZE+1:i]) for i in range(WINDOW_SIZE-1, len(validation_scores))]

# if overfit_stop and max(smoothed_val_scores) > max(smoothed_val_scores[-PATIENCE:]):
#     break

### Test!

In [ ]:
init_num = 0

gen_train = data_loader(task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE, enc_seq_len=ENC_SEQ_LEN, dec_seq_len=DEC_SEQ_LEN)
gen_val = data_loader(task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE, enc_seq_len=ENC_SEQ_LEN, dec_seq_len=DEC_SEQ_LEN)
gen_test = data_loader(task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE, enc_seq_len=ENC_SEQ_LEN, dec_seq_len=DEC_SEQ_LEN)


param = list(model_parameters)[5]
print(param)
param['enc_depth'], param['enc_heads'] = param['depth,heads']
param['dec_depth'], param['dec_heads'] = param['depth,heads']
param.pop('depth,heads')

model = XTransformer(**param).cuda()

model_name = f"{TASK_NAME}_dim{param['dim']}d{param['enc_depth']}h{param['enc_heads']}M{param['enc_num_memory_tokens']}l{param['enc_max_seq_len']}_v{init_num}"

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

src, tgt, _, _ = next(gen_train)

print(model.encoder.max_seq_len, model.encoder.num_memory_tokens)
model.encoder(torch.cat((src, src)), return_embeddings=True).shape